# Information Retrieval


이다현 TA: `leedhn@yonsei.ac.kr`

In this session, we will explore various retrieval modules.

## Import & dataset

In [1]:
!pip install datasets transformers

from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.3 MB/s eta 0:00:00


We will use SQuAD benchmark.

In [2]:
from datasets import load_dataset

dataset = load_dataset('squad')

train_dataset = load_dataset('squad', split='train[:70%]')
valid_dataset = load_dataset('squad', split='train[70%:]')
test_dataset = load_dataset('squad', split='validation')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_dataset[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [12]:
test_dataset[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [4]:
eval_corpus_set = pd.DataFrame(dataset['train']).groupby('context')  # context를 기준으로 dataframe 재구성
eval_corpus=[]
for name, document in eval_corpus_set:
    # print(document.head())
    document=document.reset_index()
    eval_corpus.append(f"{document['title'][0]} {document['context'][0]}")
# eval_corpus에는 중복 없는 title + context가 모이게 됨
len(eval_corpus)

18891

In [5]:
test_corpus_set = pd.DataFrame(dataset['validation']).groupby('context')
test_corpus=[]
for name, document in test_corpus_set:
    document=document.reset_index()
    test_corpus.append(f"{document['title'][0]} {document['context'][0]}")
test_corpus += eval_corpus  # 얜 왜하징.

random.shuffle(test_corpus)
len(test_corpus)

20958

In [6]:
eval_corpus[:5]

["2008_Summer_Olympics_torch_relay \n Australia: The event was held in Canberra, Australian Capital Territory on April 24, and covered around 16 km of Canberra's central areas, from Reconciliation Place to Commonwealth Park. Upon its arrival in Canberra, the Olympic flame was presented by Chinese officials to local Aboriginal elder Agnes Shea, of the Ngunnawal people. She, in turn, offered them a message stick, as a gift of peace and welcome. Hundreds of pro-Tibet protesters and thousands of Chinese students reportedly attended. Demonstrators and counter-demonstrators were kept apart by the Australian Federal Police. Preparations for the event were marred by a disagreement over the role of the Chinese flame attendants, with Australian and Chinese officials arguing publicly over their function and prerogatives during a press conference.",
 '2008_Summer_Olympics_torch_relay \n China: In China, the torch was first welcomed by Politburo Standing Committee member Zhou Yongkang and State Cou

## 1. Lexical method를 직접 구현해보기
  - 구현할 목록 (총 20점)
    - tf 함수 (5점)
    - idf 함수 (5점)
    - BM25 함수 (10점)

### TF & IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
# 출력된 매트릭스의 entry [i, j]의 값은 i번째 문서에서 j번째 단어가 나온 횟수를 말함.
# print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
# print(vector.vocabulary_)

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
# 1. Lexical method를 직접 구현해보기 - tf 함수
# (vector.vocabulary_에서 인덱스 t인 단어)와 (eval_corpus에서 d번째 문서)에 대한 tf 값 계산
f = np.array(vector.fit_transform(eval_corpus).toarray()).astype(int)

def tf(t, d):
  return f[d, t] / np.sum(f[d])

In [ ]:
nd = f.shape[0]
nt = f.shape[1]
tf(random.randint(nt), random.randint(nd))

In [ ]:
# 1. Lexical method를 직접 구현해보기 - idf 함수
# (vector.vocabulary_에서 인덱스 t인 단어)에 대한 idf 값 계산
n = np.sum(f > 0, axis=0)  # number of documents containing that index

def idf(t):
  return np.log((len(eval_corpus) - n[t] + 0.5) / (n[t] + 0.5) + 1).item()

In [ ]:
idf(random.randint(nt))

### BM25

https://en.wikipedia.org/wiki/Okapi_BM25

In [ ]:
# 1. Lexical method를 직접 구현해보기 - BM25 함수
# (eval_corpus에서 D번째 문서)와 쿼리 Q에 대한 BM25 score 계산
avgdl = np.mean(np.sum(f, axis=1))

def BM25(D, Q):
  k1, b = 1.2, 0.75
  Q = vector.transform([Q]).toarray()[0]
  return np.sum(
      [idf(q) * f[D, q] * (k1 + 1) / (f[D, q] + k1 * (1 - b + b * np.sum(f[D]) / avgdl))
      for q in range(len(Q)) if Q[q] > 0]
    )

In [ ]:
a = [BM25(i, query) for i in range(len(eval_corpus))]
top5 = sorted(range(len(a)), key=lambda i: a[i], reverse=True)[:5]

In [ ]:
[eval_corpus[i] for i in top5]

['Gamal_Abdel_Nasser In 1941, Nasser was posted to Khartoum, Sudan, which was part of Egypt at the time. Nasser returned to Sudan in September 1942 after a brief stay in Egypt, then secured a position as an instructor in the Cairo Royal Military Academy in May 1943. In 1942, the British Ambassador Miles Lampson marched into King Farouk\'s palace and ordered him to dismiss Prime Minister Hussein Sirri Pasha for having pro-Axis sympathies. Nasser saw the incident as a blatant violation of Egyptian sovereignty and wrote, "I am ashamed that our army has not reacted against this attack", and wished for "calamity" to overtake the British. Nasser was accepted into the General Staff College later that year. He began to form a group of young military officers with strong nationalist sentiments who supported some form of revolution. Nasser stayed in touch with the group\'s members primarily through Amer, who continued to seek out interested officers within the Egyptian Armed Force\'s various bra

## 2. Ranking에 사용되는 metric 2개 조사 및 구현
  Note) dataset에 제공되는 answer가 검색된 passage에 포함되는 경우 Positive로 판단합니다. 
  
    - Recall
    - Top-k
    - MAP
    - MRR
    - nDCG
    - etc.

### Recall

In [27]:
def recall(documents, results, answer):
  """documents: list of str(모든 문서들), results: list of int(검색 결과로 나온 문서의 인덱스들), answer: string"""
  # dataset에 제공되는 answer가 검색된 passage에 포함되는 경우 Positive로 판단합니다.
  # recall = (true positive) / (true positive + false negative)
  # recall = (relevant and retrieved) / (relevant)
  # true positive: Relevant and Retrieved
  # false negative: Relevant and Not Retrieved
  # false positive: Non-Relevant and Retrieved
  # true negative: Non-Relevant and Not Retrieved
  tp = 0
  for idx in results:
    if answer in documents[idx]: tp += 1
  tpfn = 0
  for doc in documents:
    if answer in doc: tpfn += 1
  return tp / tpfn

### Precision at k

In [28]:
def prec_at_k(k, documents, results, answer):
  """documents: list of str(모든 문서들), results: list of int(검색 결과로 나온 문서의 인덱스들), answer: string
  results는 relevance score 높은 순으로 정렬되어 있다고 가정"""
  tp = 0
  for idx in results[:k]:
    if answer in documents[idx]: tp += 1
  return tp / k

## 3. SBERT로 검색기 구현 및 2에서 구현한 성능 평가 (inference만)

    1) test_corpus embeddings 생성

    2) test_dataset의 questions 에 대한 검색 수행

    3) 2에서 구현한 metrics로 성능 평가 (평가할 passage 개수는 1,5,20,100 중에서 임의로 선택하신 후 명시해주세요)

    4) bm25 성능과 비교 분석

    5) 검색된 문서들 case study (SBERT, DPR, BM25 에 대해서 좋은 예시, 나쁜 예시 1개씩)

https://www.sbert.net/examples/applications/retrieve_rerank/README.html

```
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('model_name')

docs = ["My first paragraph. That contains information", "Python is a programming language."]
document_embeddings = model.encode(docs)

query = "What is Python?"
query_embedding = model.encode(query)
```

In [7]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9fb4f22062f75c02a6b557ad4badd087b87b21d2e8a7e6917b2bfbf360f95345
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [63]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

### 1) test_corpus embeddings 생성

In [64]:
context_embeddings = model.encode(test_corpus, batch_size=64, show_progress_bar=True)
print(context_embeddings.shape)

Batches:   0%|          | 0/328 [00:00<?, ?it/s]

(20958, 768)


### 2) test_dataset의 questions 에 대한 검색 수행

In [65]:
nq = 5
queries = test_dataset['question'][:nq]
answers = [item['text'][0] for item in test_dataset['answers'][:nq]]
query_embeddings = model.encode(queries)
print(query_embeddings.shape)

(5, 768)


In [17]:
sims = query_embeddings @ np.transpose(context_embeddings)
sims.shape  # 엔트리 [i, j]는 (i번째 질문-j번째 문서) 간의 similarity score 담고 있음

(5, 20958)

In [40]:
k = 20  # 상위 k개 결과 가져오기
results = []
for i, sim in zip(range(k), tqdm(sims)):
    rank_idx = np.argsort(sim)[::-1]
    results.append(rank_idx[:k])

100%|██████████| 5/5 [00:00<00:00, 401.56it/s]


### 3) 2에서 구현한 metrics로 성능 평가 (평가할 passage 개수 = 20)

In [41]:
# Precision: 검색 결과 중에 답이 들어 있는 문서의 비율
# Recall: 답이 들어 있는 문서들 중 검색된 비율

for i in range(nq):
  print('QUERY:', queries[i])
  print('ANSWER:', answers[i])
  for j in range(3):
    context = test_corpus[results[i][j]]
    print(f"TOP-{j+1} CONTEXT:", context[:min(len(context), 20)] + ' ...')
  print('RECALL:', recall(test_corpus, results[i], answers[i]))
  print('PRECISION AT 10:', prec_at_k(10, test_corpus, results[i], answers[i]))
  print('-' * 10)

QUERY: Which NFL team represented the AFC at Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 The Br ...
TOP-3 CONTEXT: Super_Bowl_50 The le ...
RECALL: 0.5
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team represented the NFC at Super Bowl 50?
ANSWER: Carolina Panthers
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 Caroli ...
TOP-3 CONTEXT: Super_Bowl_50 The le ...
RECALL: 0.75
PRECISION AT 10: 0.1
----------
QUERY: Where did Super Bowl 50 take place?
ANSWER: Santa Clara, California
TOP-1 CONTEXT: Super_Bowl_50 The le ...
TOP-2 CONTEXT: Super_Bowl_50 On May ...
TOP-3 CONTEXT: Super_Bowl_50 The ga ...
RECALL: 1.0
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team won Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 The Br ...
TOP-3 CONTEXT: Super_Bowl_50 The Br ...
RECALL: 0.25
PRECISION AT 10: 0.0
----------
QUERY: What color was used to emph

### 4) bm25 성능과 비교 분석

In [37]:
!pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split() for doc in test_corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [43]:
for i in range(nq):
  print('QUERY:', queries[i])
  print('ANSWER:', answers[i])

  tokenized_query = queries[i].split()
  bm25_scores = bm25.get_scores(tokenized_query)
  rank_idx = np.argsort(bm25_scores)[::-1]
  result = rank_idx[:k]

  for j in range(3):
    context = test_corpus[result[j]]
    print(f"TOP-{j+1} CONTEXT:", context[:min(len(context), 20)] + ' ...')
  print('RECALL:', recall(test_corpus, result, answers[i]))
  print('PRECISION AT 10:', prec_at_k(10, test_corpus, result, answers[i]))
  print('-' * 10)

QUERY: Which NFL team represented the AFC at Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Super_Bowl_50 As the ...
TOP-2 CONTEXT: Super_Bowl_50 The Pa ...
TOP-3 CONTEXT: Super_Bowl_50 On May ...
RECALL: 0.5
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team represented the NFC at Super Bowl 50?
ANSWER: Carolina Panthers
TOP-1 CONTEXT: Super_Bowl_50 As the ...
TOP-2 CONTEXT: Super_Bowl_50 The Pa ...
TOP-3 CONTEXT: Super_Bowl_50 On May ...
RECALL: 0.5
PRECISION AT 10: 0.1
----------
QUERY: Where did Super Bowl 50 take place?
ANSWER: Santa Clara, California
TOP-1 CONTEXT: Super_Bowl_50 CBS br ...
TOP-2 CONTEXT: Super_Bowl_50 The Su ...
TOP-3 CONTEXT: Super_Bowl_50 On Jun ...
RECALL: 1.0
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team won Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Seattle The Seahawks ...
TOP-2 CONTEXT: Super_Bowl_50 On May ...
TOP-3 CONTEXT: Super_Bowl_50 As the ...
RECALL: 0.5
PRECISION AT 10: 0.1
----------
QUERY: What color was used to emphas

### 5) 검색된 문서들 case study (SBERT, DPR, BM25 에 대해서 좋은 예시, 나쁜 예시 1개씩)

DPR 이용 검색

In [44]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder
from transformers import DPRConfig

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
configuration = DPRConfig()
qe = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
ctxe = DPRQuestionEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['ctx_encoder.bert_model.encoder.layer.4.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.0.intermediate.dense.bias', 'ctx_encoder.bert_model.encoder.layer.9.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.1.attention.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.1.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.4.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.4.intermediate.dense.bias', 'ctx_encoder.bert_model.encoder.layer.8.intermediate.dense.bias', 'ctx_encoder.bert_model.encoder.layer.3.attention.self.key.weight', 'ctx_encoder.bert_model.encoder.layer.8.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.3.attention.self.value.bias', 'ctx_encoder.bert_model.encoder.layer.9.attention.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.0.attention.o

In [45]:
import torch
torch.cuda.is_available()

True

In [48]:
q_embeddings=[]
qe.to('cuda')
ctxe.to('cuda')

for query in tqdm(queries):
    q_input_ids = tokenizer(query, return_tensors="pt",padding=True, truncation=True)["input_ids"]
    q_embedding = qe(q_input_ids.to('cuda')).pooler_output
    q_embedding = q_embedding.cpu()
    q_embeddings.append(q_embedding.detach().numpy())
    del q_embedding

q_embeddings = np.array(q_embeddings)
q_embeddings.shape

100%|██████████| 5/5 [00:00<00:00, 53.00it/s]


(5, 1, 768)

In [49]:
ctx_embeddings=[]
for ctx in tqdm(test_corpus):
    ctx_input_ids = tokenizer(ctx, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    ctx_embedding = ctxe(ctx_input_ids.to('cuda:0')).pooler_output
    ctx_embedding = ctx_embedding.cpu()
    ctx_embeddings.append(ctx_embedding.detach().numpy())
    del ctx_embedding

100%|██████████| 20958/20958 [11:06<00:00, 31.44it/s]


In [50]:
ctx_embeddings_np = np.array(ctx_embeddings)
q_embeddings.shape, ctx_embeddings_np.shape

((5, 1, 768), (20958, 1, 768))

In [55]:
sims = np.dot(np.squeeze(q_embeddings, axis=1),np.transpose(np.squeeze(ctx_embeddings_np, axis=1)))
sims.shape

(5, 20958)

In [58]:
for i in range(nq):
  print('QUERY:', queries[i])
  print('ANSWER:', answers[i])

  rank_idx = np.argsort(sims[i])[::-1]
  result = rank_idx[:k]

  for j in range(3):
    context = test_corpus[result[j]]
    print(f"TOP-{j+1} CONTEXT:", context[:min(len(context), 20)] + ' ...')
  
  print('RECALL:', recall(test_corpus, result, answers[i]))
  print('PRECISION AT 10:', prec_at_k(10, test_corpus, result, answers[i]))
  print('-' * 10)

QUERY: Which NFL team represented the AFC at Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Paper Most commercia ...
TOP-2 CONTEXT: Imperialism Imperial ...
TOP-3 CONTEXT: Wood Knots do not ne ...
RECALL: 0.0
PRECISION AT 10: 0.0
----------
QUERY: Which NFL team represented the NFC at Super Bowl 50?
ANSWER: Carolina Panthers
TOP-1 CONTEXT: Imperialism Imperial ...
TOP-2 CONTEXT: Paper Most commercia ...
TOP-3 CONTEXT: Pesticide Pesticides ...
RECALL: 0.0
PRECISION AT 10: 0.0
----------
QUERY: Where did Super Bowl 50 take place?
ANSWER: Santa Clara, California
TOP-1 CONTEXT: Wood Wood density is ...
TOP-2 CONTEXT: Wood The general eff ...
TOP-3 CONTEXT: Wayback_Machine As o ...
RECALL: 0.0
PRECISION AT 10: 0.0
----------
QUERY: Which NFL team won Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Catalan_language Lin ...
TOP-2 CONTEXT: Somalis In the Unite ...
TOP-3 CONTEXT: Pesticide Pesticides ...
RECALL: 0.0
PRECISION AT 10: 0.0
----------
QUERY: What color was used to emphas

## 4. 검색 모델의 성능 향상시킬 수 있는 방법 탐색
  아래의 예시 중에서 하나를 선택해서 탐색하고 1~2문단 내외로 간략하게 적어주세요.
  
  논문을 읽고 소개하셔도 좋고 (티스토리랑 velog는 ...티 안 나게..ㅎ.ㅎ), 

  더 발전된 검색 모델을 찾아서 직접 검색을 수행해봐도 됩니다. 

  예시)
  - 다른 모델과의 앙상블 (e.g., BM25)
    - e.g., add scores
    - 어떻게 앙상블을 수행할 수 있을지?

  - data augmentation (query generation)
    - https://arxiv.org/abs/2204.07496
    - 어떻게 data augmentation을 할 수 있을지? 
    - 다른 데이터셋도 같이 학습해 보는 건?
    - 검색 task에는 어떤 데이터셋들이 있을지?
    - 문제점은 무엇인지?


  - DPR training with hard negatives
    - https://arxiv.org/abs/2004.04906
    - hard negative가 뭔지, DPR 학습이 어떻게 이루어지는지?
    - batch size가 어떤 영향을 미칠지?

  - Use advanced models
    - https://huggingface.co/sebastian-hofstaetter/colberter-128-32-msmarco
    - https://huggingface.co/facebook/contriever
    - https://huggingface.co/Luyu/co-condenser-marco-retriever
    - 어떤 모델들이 있는지?

  - etc. (자유 주제)

## 5. 검색이 실제 서비스에 제공되려면?
  - 실제로 검색이 수행되는 환경은, 어마어마한 크기의 Web-corpus domain입니다. 아래 질문에 간략하게 답해주세요. 


     1) 왜 query encoder, passage encoder 로 분리해둔 이유는 무엇인지? 특히, query-passage concat으로 검색을 수행할 때 생기는 문제점은?
        hint: Bi-encoder, Cross-encoder, inference time, computational cost

     2) 관련 있는 문서를 찾는데 걸리는 시간을 줄이기 위한 방법으로는 어떤게 있을지?


  
  - 5-2)에서 해주신 답 중 하나를 실제로 구현한 게 `faiss` 입니다! 

    SBERT로 만들었던 corpus representation들로 `faiss index`를 만들어보고, 
    
    `faiss`의 `k-nearest neighbor search`로 얻은 검색결과와 앞서 얻은 검색 결과를 비교해주세요. 짧게!

In [61]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.1 MB/s eta 0:00:00


In [66]:
import faiss

# 실행 이전 SBERT context_embeddings, query_embeddings 계산하는 부분 먼저 실행
context_embeddings = np.array(context_embeddings)
query_embeddings = np.array(query_embeddings)
d = context_embeddings.shape[1]

In [67]:
Index = faiss.IndexFlatL2(d)
Index.add(context_embeddings)
print(Index.ntotal)

20958


In [68]:
k = 20
distances, indexes = Index.search(query_embeddings, k)
print(distances)

[[0.7136224  0.7395483  0.77168965 0.7730047  0.7867346  0.79998034
  0.8104599  0.82063097 0.82641965 0.8334012  0.85414004 0.8572709
  0.8629759  0.86466044 0.8675992  0.8755057  0.8793607  0.88493717
  0.88690114 0.8923301 ]
 [0.6867496  0.76819783 0.7987458  0.8440975  0.84589326 0.8478636
  0.8484086  0.85615265 0.8603953  0.86672515 0.87908524 0.8841291
  0.88742095 0.8920749  0.8931496  0.89697    0.90732837 0.9073904
  0.9176087  0.919725  ]
 [0.54936606 0.61826456 0.65649366 0.6992655  0.7752725  0.78174955
  0.8147966  0.82469314 0.83811164 0.8496592  0.85715574 0.85722935
  0.8616726  0.8828584  0.8960759  0.89725727 0.89935446 0.9059128
  0.90788394 0.91087663]
 [0.58702606 0.6726596  0.708579   0.73668766 0.7410375  0.74832994
  0.7628596  0.7643249  0.76677173 0.7775836  0.7816522  0.78503
  0.78938967 0.7907209  0.8156072  0.82290214 0.8262691  0.8409177
  0.84643817 0.84775686]
 [0.58803284 0.72435844 0.8039044  0.8083539  0.8350805  0.84127325
  0.8860627  0.89784896 0

In [69]:
for i in range(nq):
  print('QUERY:', queries[i])
  print('ANSWER:', answers[i])

  result = indexes[i]

  for j in range(3):
    context = test_corpus[result[j]]
    print(f"TOP-{j+1} CONTEXT:", context[:min(len(context), 20)] + ' ...')
  
  print('RECALL:', recall(test_corpus, result, answers[i]))
  print('PRECISION AT 10:', prec_at_k(10, test_corpus, result, answers[i]))
  print('-' * 10)

QUERY: Which NFL team represented the AFC at Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 The Br ...
TOP-3 CONTEXT: Super_Bowl_50 The le ...
RECALL: 0.5
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team represented the NFC at Super Bowl 50?
ANSWER: Carolina Panthers
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 Caroli ...
TOP-3 CONTEXT: Super_Bowl_50 The le ...
RECALL: 0.75
PRECISION AT 10: 0.1
----------
QUERY: Where did Super Bowl 50 take place?
ANSWER: Santa Clara, California
TOP-1 CONTEXT: Super_Bowl_50 The le ...
TOP-2 CONTEXT: Super_Bowl_50 On May ...
TOP-3 CONTEXT: Super_Bowl_50 The ga ...
RECALL: 1.0
PRECISION AT 10: 0.1
----------
QUERY: Which NFL team won Super Bowl 50?
ANSWER: Denver Broncos
TOP-1 CONTEXT: Super_Bowl_50 The Pa ...
TOP-2 CONTEXT: Super_Bowl_50 The Br ...
TOP-3 CONTEXT: Super_Bowl_50 The Br ...
RECALL: 0.25
PRECISION AT 10: 0.0
----------
QUERY: What color was used to emph

# **보고서** (총 80점)

  - 2번의 metric 2개 설명 (10점)
  - SBERT 결과(10점), bm25와의 성능 비교(5점), case study(15점) (총 30점)
  - 탐색한 성능향상 방법 (20점)
  - faiss 질문 2개 답변(10점), faiss index 구현 및 결과 (10점) (총 20점)
  - 한/영 상관 없음
  - max 4페이지, *간략하게* 기술하여 pdf로 제출

  
# 제출할 colab 파일 (총 20점)
  - 1번 lexical method 3가지 구현 코드 및 성능
    - 수행 결과는 위에 실습했던 샘플과 동일한 데이터로 진행해주시면 됩니다.
  - 2번 metric 구현 코드